In [ ]:
"""
A simple test of Ray
This example uses placement_group API to spread work around
"""

import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()

    def ping(self):
        print(f"{self.pid} {self.hostname} {self.ip} - ping")
        time.sleep(1)
        return f"{self.pid} {self.hostname} {self.ip} - pong"

@ray.remote
def main():
   
    # Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")

    # Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    actor = []
    for a in actors:
        node_ip_str = f"node:{ray._private.services.get_node_ip_address()}"
        actor.append(Actor.options(resources={f"{node_ip_str}": 1}).remote())
    time.sleep(1)

    # Ping-Pong test
    for _ in range(5):
        messages = [a.ping.remote() for a in actors]
        for msg in ray.get(messages):
            time.sleep(1)
            print(f"Received back message {msg}")
            time.sleep(1)

            
actors = [Actor.remote() for _ in range(1)]

if __name__ == "__main__":
    main.remote()